In [19]:
import os
import time
import pandas as pd
import pymysql
import numpy as np
from tqdm.auto import tqdm

def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# 숙박업소 주변 맛집 인기 메뉴 정보

# RT_LDGBSSH_CFR_GBRSTRNT_POPULAR_MENU_INFO_202407

## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=10de8bc0-2642-11ec-bbc0-d7035fffebeb

In [13]:
def text_extract(input_text):
    """상세주소 제외한 함수"""
    result_text = " ".join(input_text.split()[:-1])
    if result_text.endswith("리"):
        return " ".join(result_text.split()[:-1])
    return result_text

with dev_db_conn() as conn:
    sql = """
    SELECT T1.id, T1.name, T1.road_addr, T1.common_addr, T1.latitude, T1.longitude, T1.category
    FROM redtable_raw_2022.naver_other T1
    WHERE T1.category IN ('호텔', '모텔', '게스트하우스', '펜션');
    """
    df_hotel = pd.read_sql(sql, conn)
    
df_hotel["검색주소"] = df_hotel["common_addr"].apply(lambda x:text_extract(x))

# 임의의 id 생성 (RSTR 테이블의 마지막 ID값으로 더해서 생성)
df_hotel["hotel_id"] = df_hotel.index + 300680045

In [14]:
df_hotel

,id,name,road_addr,common_addr,latitude,longitude,category,검색주소,hotel_id
0,11583212,롯데호텔 월드,서울 송파구 올림픽로 240,서울특별시 송파구 잠실동 40-1,37.511418,127.099836,호텔,서울특별시 송파구 잠실동,300680045
1,11595764,호텔선샤인 서울,서울 강남구 도산대로 205 선샤인 관광호텔,서울특별시 강남구 신사동 587-1 선샤인 관광호텔,37.520626,127.028847,호텔,서울특별시 강남구 신사동 587-1 선샤인,300680046
2,11657754,상대온천관광호텔,,경상북도 경산시 남산면 상대리 590,35.780875,128.792400,호텔,경상북도 경산시 남산면,300680047
3,11658968,시티팔레스 관광호텔,서울 동대문구 답십리동 천호대로 257,서울특별시 동대문구 답십리동 497-23,37.569119,127.050514,호텔,서울특별시 동대문구 답십리동,300680048
4,11659045,파레스관광호텔,광주 동구 중앙로160번길 13 파레스관광호텔,광주광역시 동구 황금동 11-4,35.147700,126.914635,호텔,광주광역시 동구 황금동,300680049
...,...,...,...,...,...,...,...,...,...
94,1818756615,브라운도트 영도대교점,부산 영도구 절영로19번길 37 브라운도트 호텔,부산광역시 영도구 대교동2가 32-3 브라운도트 호텔,35.092002,129.041928,호텔,부산광역시 영도구 대교동2가 32-3 브라운도트,300680139
95,1852221284,오션스파호텔,경남 고성군 고성읍 신월로 60,경상남도 고성군 고성읍 신월리 589,34.952627,128.331097,호텔,경상남도 고성군 고성읍,300680140
96,1858668986,하운드호텔 대구성서,대구 달서구 성서공단로11길 44-9,대구광역시 달서구 호림동 10-11,35.839164,128.489161,호텔,대구광역시 달서구 호림동,300680141
97,1914435701,노블 스테이 호텔,서울 송파구 올림픽로12길 4-8 노블스테이호텔,서울특별시 송파구 잠실동 181-20 노블스테이호텔,37.511311,127.082469,호텔,서울특별시 송파구 잠실동 181-20,300680142


In [17]:
for i in tqdm(range(len(df_hotel))):
    with dev_db_conn() as conn:
        with conn.cursor() as curs:
            sql = f"""
            SELECT DISTINCT T4.PRODUCT_ID AS 'MENU_ID', %s AS 'HOTEL_ID', %s AS 'HOTEL_NM',
               T3.SRC_ID AS 'RSTRNT_ID', T1.RSTR_NM AS 'RSTRNT_NM', T4.MENU_NM, T4.MENU_PRICE AS 'MENU_PRC',
                GROUP_CONCAT(DISTINCT IF(T5.TYPE_NM = '주재료', T6.TAG_NM, NULL)) AS 'MAIN_INGREDIENT_DC',
                GROUP_CONCAT(DISTINCT IF(T5.TYPE_NM = '조리법', T6.TAG_NM, NULL)) AS 'CKNG_STYLE_DC',
                GROUP_CONCAT(DISTINCT IF(T5.TYPE_NM = '소스', T6.TAG_NM, NULL)) AS 'SOURC_DC',
                GROUP_CONCAT(DISTINCT IF(T5.TYPE_NM = '옵션', T6.TAG_NM, NULL)) AS 'OPTN_DC'
            FROM redtable01.RSTR T1
            INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
            INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
            INNER JOIN (SELECT RSTR_ID, PRODUCT_ID, MENU_NM, MENU_PRICE, ID
                FROM redtable01.MENU
                WHERE PRODUCT_ID IS NOT NULL AND PRODUCT_ID<>'' AND REPRSNT_MENU_YN = 'Y'
                GROUP BY MENU_NM) T4 ON T3.RSTR_ID = T4.RSTR_ID
            LEFT JOIN redtable01.MENU_TAG T5 ON T4.ID = T5.MENU_ID
            LEFT JOIN redtable01.MENU_TAG_MASTER T6 ON T5.MENU_TAG_MASTER_ID = T6.ID
            WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
                AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%%？%%'
                AND T1.RSTR_LNNO_ADRES LIKE %s
            GROUP BY T4.PRODUCT_ID
            """
            curs.execute(sql, (df_hotel["hotel_id"][i], df_hotel["name"][i], f'%{df_hotel["검색주소"][i]}%'))
            df_result = pd.DataFrame(curs.fetchall())
            
    if i == 0:
        df_final = df_result
    
    else:
        df_final = pd.concat([df_final, df_result])
        
# 결측치 및 불필요한 데이터 정리
df_final = df_final.fillna(np.nan)
df_final.replace({np.nan:None, '\\n':None}, inplace=True)

  0%|          | 0/99 [00:00<?, ?it/s]

In [18]:
df_final

,MENU_ID,HOTEL_ID,HOTEL_NM,RSTRNT_ID,RSTRNT_NM,MENU_NM,MENU_PRC,MAIN_INGREDIENT_DC,CKNG_STYLE_DC,SOURC_DC,OPTN_DC
0,58055,300680045,롯데호텔 월드,12390,알루메,토마토 해산물 리조또,17000.00,"토마토,해산물",None,None,None
1,58056,300680045,롯데호텔 월드,12390,알루메,토마토새우 리조또,15000.00,"새우,토마토",None,None,None
2,58057,300680045,롯데호텔 월드,12390,알루메,크림 버섯 리조또,16000.00,"버섯,쌀,야채,크림",리소토,None,"사이드메뉴,추가"
3,58058,300680045,롯데호텔 월드,12390,알루메,크림 게살 리조또,16000.00,"게살,계란,김,쌀,야채,크림","리소토,볶음,부침",None,추가
4,58059,300680045,롯데호텔 월드,12390,알루메,오일 치킨 라이스,16000.00,"닭고기,밀가루,쌀,야채","볶음,치킨",None,None
...,...,...,...,...,...,...,...,...,...,...,...
37,2823962,300680141,하운드호텔 대구성서,21049,현풍닭칼국수,삼계칼국수,9500.00,"면,야채",칼국수,None,None
38,3429086,300680141,하운드호텔 대구성서,21145,제주신물회,제주스페셜A(1인),50000.00,"랍스터,소등심,폭립",구이,None,스페셜
39,3429087,300680141,하운드호텔 대구성서,21145,제주신물회,제주스페셜B(1인),40000.00,"생선,생선회",None,None,스페셜
40,3429088,300680141,하운드호텔 대구성서,21145,제주신물회,제주스페셜C(1인),35000.00,돼지고기,삶은,간장,스페셜


In [15]:
df_final2 = df_final

In [ ]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_LDGBSSH_CFR_GBRSTRNT_POPULAR_MENU_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [ ]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT IGNORE INTO redtable_bigdata_center.RT_LDGBSSH_CFR_GBRSTRNT_POPULAR_MENU_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df_final.apply(lambda row:(row["MENU_ID"], row["HOTEL_ID"], row["HOTEL_NM"], row["RSTRNT_ID"],
                                  row["RSTRNT_NM"], row["MENU_NM"], row["MENU_PRC"], row["MAIN_INGREDIENT_DC"],
                                  row["CKNG_STYLE_DC"], row["SOURC_DC"], row["OPTN_DC"]), axis=1).tolist()
        curs.executemany(sql, val)